https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

In [1]:
import pandas as pd
import numpy as np

### Reading in the Data

In [2]:
test_data = pd.read_csv('test_set.csv')
train_data = pd.read_csv('train_set.csv')

In [3]:
train_data

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [4]:
#example of what you can do to remove the Timezone when it is given (case REG_DATE)
#pd.to_datetime(sorted_train.iloc[0,2], dayfirst=True).tz_localize(None)


### The code to get the remaining time untill the end of the case 

In [5]:
def get_last_times(data, case_col:str, last_time_col:int):
    '''Function to make a dictionary with the final times of each case
    '''
    cases = data[case_col].unique()
    #cases = data.iloc[:, case_col].unique() if case_col:int
    
    last_times_list = []
    for i in range(len(cases)):
        last_time = data[data[case_col] == cases[i]].copy().iloc[[-1], last_time_col].values[0]
        last_times_list.append(last_time)
        
    last_times = dict(zip(cases, last_times_list))
        
    return last_times

In [6]:
def time_to_end(data, case_col_str:str, case_col:int, index_col_str:str, time_col:int):
    '''Function to calculate the time remaining to the end of a case
    '''
    sorted_data = data.sort_values(by=[case_col_str, index_col_str])
    
    last_times = get_last_times(sorted_data, case_col_str, time_col)
    
    till_end = []
    for i in range(len(data)):
        till_end.append(pd.to_datetime(last_times[data.iloc[i,case_col]], dayfirst=True) - 
                        pd.to_datetime(data.iloc[i,time_col], dayfirst=True))
    res = data.copy()    
    res['Time till end of case'] = till_end
    return res

In [7]:
#running this cell, will take a long time (depending on computer: 2-5 min)
train_data_endtime = time_to_end(train_data, 'case concept:name', 1, 'eventID ', 6)

In [8]:
train_data_endtime

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Time till end of case
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546,12 days 09:58:52.480000
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880,12 days 09:58:52.146000
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906,12 days 09:57:59.120000
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875,12 days 09:57:58.151000
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256,9 days 06:08:36.377000
...,...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361,0 days 00:00:03.826000
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361,0 days 00:00:03.826000
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187,0 days 00:00:00
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299,0 days 00:04:18.932000
